Step 1:

This notebook prepares the Ontonotes data for the model.  It contains a few steps:

+ Copy files from google drive location to colab location
+ Parse CoNLL format files into csv's containing the phrase number
+ Save csv's to colab location

Once run, the data is saved for future uses at \root\data\parsed (or file indicated at output_dir).  This seems to be a user dependent directory so will have to be run for each person.  Also the drive_location might need to be changed to the ID for the google drive folder per user.  These files are 

TODO:
+ Find a file location that is shareable across users


Step 2:
Consume the ontonotes csv's prepared earlier.  Generate embeddings for use in the model.

We want to use multiple embedding sources:
+ GloVe
+ CoVe
+ BERT

Each of these embedding sources has different vector lengths.  In the case of BERT we need to tokenize data according to how the model expects it, which will probably require a separate tokenizer as well.  Separate models will likely have to be created for each of the embedding sources.

Tokenizing will strip out useful information such as:
+ uppercase / lowercase / titlecase

This information is useful for NER.  We want to preserve this information and attach it to the embeddings.

We'll start by focusing on GloVe since it's pretty simple.

# Configuration

In [1]:
pip install --upgrade wandb

     |████████████████████████████████| 1.3MB 2.8MB/s 
     |████████████████████████████████| 92kB 24.2MB/s 
     |████████████████████████████████| 460kB 39.5MB/s 
     |████████████████████████████████| 81kB 27.4MB/s 
     |████████████████████████████████| 235kB 40.5MB/s 
     |████████████████████████████████| 102kB 29.1MB/s 
     |████████████████████████████████| 71kB 23.5MB/s 
     |████████████████████████████████| 409kB 47.7MB/s 
     |████████████████████████████████| 256kB 43.5MB/s 
     |████████████████████████████████| 184kB 45.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/61/1d/d0/04cfe495619be2095eb8d89a31c42adb4e42b76495bc8f784c
  Stored in directory: /root/.cache/pip/wheels/3f/eb/fd/69e5177f67b505e44acbd1aedfbe44b91768ee0c4cd5636576
  Stored in directory: /root/.cache/pip/wheels/a7/8d/65/a3247f500d675d80a01e4d2f0ee44fe99f1faef575bc2a1664
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Stored in dire

In [2]:
import os
import time
import sys
import csv
import pandas as pd
import numpy as np
import string
import ast
from IPython.display import display

from shutil import copyfile
from google.colab import drive
drive.mount('/content/drive')

csv.field_size_limit(sys.maxsize)

from keras.layers import TimeDistributed, Conv1D, Dense, Embedding, Input, Dropout, LSTM, Bidirectional, MaxPooling1D, Flatten, concatenate
from keras.initializers import RandomUniform
from keras.optimizers import SGD, Nadam
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, EarlyStopping

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
print('Found GPU at: {}'.format(device_name))

import wandb
from wandb.keras import WandbCallback

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Using TensorFlow backend.


Found GPU at: /device:GPU:0


In [3]:
### File configurations
drive_dir = "/content/drive/My Drive/W266_Project/"
data_src = os.path.join(drive_dir,"ontonotes")
embed_src = os.path.join(drive_dir,"embeddings")

# cache store
cache_dir = os.path.join(drive_dir, "cache")
embed_store =  os.path.join(cache_dir, 'embed.h5')

### Model Parameters
wandb.init(project="w266-final", name="your custom run name")
config = wandb.config # Config is a variable that holds and saves hyperparameters and inputs
config.dropout = 0.2
config.recurrent_dropout = 0.25
config.char_vocab = len(string.printable)
config.char_embedding_dim = 30
config.word_length = 52
config.conv_size = 3
config.conv_filters = 30
config.conv_stride = 1
config.conv_window = 52
config.lstm_state_size = 200

config.epochs = 20
config.batch_size = 400
config.training_size = 900000

# embedding to use
# 50d vector is consistent with paper
embedding_file = "glove.6B.50d.txt"

# training data
train_file = 'onto.train.ner'
dev_file = 'onto.development.ner'
test_file = 'onto.test.ner'

model_dir = os.path.join(drive_dir, 'output')

# if loading a pre-trained model set these
model_name = "std_400b_glove50d_full_04-0.0105.h5"
model_load_path = os.path.join(model_dir, model_name + '.h5')
INITIAL_EPOCH = 4

# else use these
model_name = "std_400b_glove50d_full"
model_path = os.path.join(model_dir, model_name + '_{epoch:02d}-{val_loss:.4f}.h5')
model_results_path = os.path.join(model_dir, model_name + '.csv')


### Overwrite and refresh saved files
# enable these when changing part of the pre-processing routines
# by default we don't pre-process each time for performance reasons
OVWR_ONTO = False
OVWR_DATA = False
OVWR_EMBED = False

### Preprocessing Parameters
UNK_WORD = "<UNK-WORD>"
PAD_WORD = "<PAD-WORD>"

UNK_CHAR = "<UNK-CHAR>"
PAD_CHAR = "<PAD-CHAR>"

# max number of words in a sentence, pad to this length, might throw an error if the sentence is longer
SENTENCE_WIDTH = 256
# max number of characters in a word, pad to this length, will truncate if word is too long
WORD_WIDTH = 52
# symbols to map padding to
CHAR_PAD_SYMBOL = PAD_CHAR
LABEL_PAD_SYMBOL = 'O'
CASE_PAD_SYMBOL = 'other'

<IPython.core.display.Javascript object>

Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
!wandb login 235eba5562b3e4de74ca58d14b2ff8b058cb9986

Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


# Setup

In [5]:
# show files
print(os.listdir(data_src))
print(os.listdir(embed_src))
print(os.listdir(cache_dir))

['archive', 'onto.development.ner.sample', 'onto.development.ner', 'onto.test.ner.sample', 'onto.train.ner.sample', 'onto.test.ner', 'onto.train.ner']
['glove.6B.100d.txt', 'glove.6B.200d.txt', 'glove.6B.300d.txt', 'glove.6B.50d.txt', 'readme.md', 'Skip100']
['onto_train_nerword.npy', 'onto_train_nerchar.npy', 'onto_train_nercase.npy', 'onto_train_nerlabel.npy', 'onto_development_nerword.npy', 'onto_development_nerchar.npy', 'onto_development_nercase.npy', 'onto_development_nerlabel.npy', 'onto_test_nerword.npy', 'onto_test_nerchar.npy', 'onto_test_nercase.npy', 'onto_test_nerlabel.npy', 'embed.h5']


# Preprocessing

## Load Embeddings

In [0]:
# consider using tf.nn.embedding_lookup instead
# or maybe nltk.tokenize


def get_casing_ix(word):
  '''
  determines the casing of the word
  
  returns casing_ix
  '''
  if word.istitle():
    return case_to_ix['title']
  elif word.islower():
    return case_to_ix['lower']
  elif word.isupper():
    return case_to_ix['upper']
  elif word.isnumeric():
    return case_to_ix['numeric']
  return case_to_ix['other']

def get_word_ix(word):
  '''
  takes w and returns the index of the word embedding
  out of vocabulary terms return the UNK_WORD and the character embeddings
  
  returns word_ix
  '''
  w = word.lower()
  w_ix = word_to_ix.get(w)
  if w_ix is not None:
    return w_ix
  return word_to_ix[UNK_WORD]

def get_char_ix(char):
  char_ix = char_to_ix.get(char)
  if char_ix is not None:
    return char_ix
  return char_to_ix[UNK_CHAR]
  
def create_character_embeddings(words_df):
  '''
  Optional function to create pre-trained character embeddings from averaged word embeddings.  In the model we generate them from a uniform random distribution and train.
  '''
  characters = {}
  for i, word_vec in enumerate(words_df.reset_index().values):
    for char in word_vec[0]:
      if char in characters:
        characters[char] = [characters[char][0] + word_vec[1:].astype(float), characters[char][1] + 1]
      else:
        characters[char] = [word_vec[1:].astype(float), 1]

  for key in characters:
    characters[key] = np.round(characters[key][0]/characters[key][1],6)
    
def initialize_word_embeddings(file_name, use_cache=True, debug=True, save_cache=True):
  loaded = False
  df = None
  
  if use_cache:
    try:
      print("Attempting to load from cache")
      with pd.HDFStore(embed_store, 'r') as store:
        words = store[file_name]
      words = pd.read_hdf(embed_store, file_name)
      loaded=True
      print("Loaded successfully")
    except:
      print("Cache loading failed")
      loaded=False
  
  if not loaded:
    words = pd.read_csv(os.path.join(embed_src, embedding_file), sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)
    # some embeddings come back with word == NaN
    words = words[~words.index.isnull()]
    # add entries for special tokens
    words.loc[UNK_WORD] = [0 for x in words.columns]
    words.loc[PAD_WORD] = [0 for x in words.columns]
    if save_cache:
      with pd.HDFStore(embed_store, 'a') as store:
        store[file_name] = words
  
  word2ix = {word:i for i,word in enumerate(words.index)}
  ix2word = {i:word for i,word in enumerate(words.index)}
  words = words.to_numpy().astype(float)
  
  return words, word2ix, ix2word

def initialize_character_embeddings(vocab=string.printable):
  characters = [x for x in string.printable]
  characters += [UNK_CHAR, PAD_CHAR]
  char2ix = {ch:i for i, ch in enumerate(characters)}
  ix2char = {i:ch for i, ch in enumerate(characters)}
  
  return characters, char2ix, ix2char

def initialize_case_embeddings(vocab=['upper','lower','title','numeric','other']):
  case2ix = {case:i for i, case in enumerate(vocab)}
  ix2case = {}
  cases = []
  for k,v in case2ix.items():
    this_case = np.zeros(len(case2ix))
    this_case[v] = 1
    cases.append(this_case)
    ix2case[v] = k
  cases = np.array(cases)
  
  return cases, case2ix, ix2case

  
def initialize_labels(file_name):
  data = pd.read_csv(os.path.join(data_src, file_name), sep="\t",  quoting=csv.QUOTE_NONE, header=None, skip_blank_lines=False, engine='python', names =['token', 'pos', 'tree', 'BIO'])
  data.dropna(subset=['BIO'], inplace=True)
  label_list = data.BIO.unique()
  label2ix = {label:i for i, label in enumerate(label_list)}
  ix2label = {i:label for i, label in enumerate(label_list)}
  return label_list, label2ix, ix2label

In [7]:
# load embeddings and format
words, word_to_ix, ix_to_word = initialize_word_embeddings(embedding_file, use_cache=True)
characters, char_to_ix, ix_to_char = initialize_character_embeddings()
cases, case_to_ix, ix_to_case = initialize_case_embeddings()
labels, label_to_ix, ix_to_label = initialize_labels(train_file)

Attempting to load from cache
Loaded successfully


## Process Data

In [0]:
def checkPrior(blah):
  if blah is None:
    return True
  else:
    return False
  
def phrase2char(w_vec):
  '''
  This function transforms a sequence of words in index format to a 2d array of character indexes
  
  w_vec - an iterable of word indexes
  
  returns np.ndarray of size (len(w_vec), WORD_WIDTH)
  '''
  phrase_vector = []
  for w_ix in w_vec:
    char_vector = []
    if w_ix not in (word_to_ix[PAD_WORD],word_to_ix[UNK_WORD]):
      for char in ix_to_word[w_ix]:
        char_vector.append(get_char_ix(char))
    phrase_vector.append(np.array(char_vector))
  return pad_sequences(phrase_vector, value=char_to_ix[PAD_CHAR], maxlen=WORD_WIDTH, padding='post')

def pad_truncate(x,width,pad_token):
  if(len(x) > width):
    print(f"Truncating input: {[ix_to_word[ix] for ix in x]}")
    x = x[:256]
  return np.pad(x,pad_width=(0,width-len(x)), mode='constant', constant_values=pad_token)

def verbosity(str, verbose):
  if verbose:
    print(str)

def preprocess_data(file_name, use_cache=True, debug=True):
  '''
  Prepares data for model.  It can be used for both training and test data.
  
  returns pd.DataFrame
  '''
  clean_name = os.path.join(cache_dir, file_name.replace(".", "_"))
  loaded = False
  phrase_vectors = None
      
  if use_cache and os.path.exists(clean_name+"word.npy"):
    verbosity("Attempting to load from cache", debug)
    try:
      word_vectors = np.load(clean_name+"word.npy", allow_pickle=True)
      char_vectors = np.load(clean_name+"char.npy", allow_pickle=True)
      case_vectors = np.load(clean_name+"case.npy", allow_pickle=True)
      label_vectors = np.load(clean_name+"label.npy", allow_pickle=True)
      phrase_vectors = [word_vectors, char_vectors, case_vectors, label_vectors]
      loaded = True
      verbosity("Loaded successfully", debug)
    except:
      verbosity("Loading failed",debug)
      loaded = False
  
  if not loaded:
    verbosity(f"Loading raw data file to process labels: {file_name}", debug)
    checkpoint = time.time()  
    data = pd.read_csv(os.path.join(data_src, file_name), sep="\t",  quoting=csv.QUOTE_NONE, header=None, skip_blank_lines=False, engine='python', names =['token', 'pos', 'tree', 'BIO'])
    verbosity(f"Parsed data loaded: {time.time()-checkpoint} s", debug)

    # see if prior row was a newline
    data['prior'] = data.token.shift(1)
    # drop empty rows
    data = data.loc[~data.token.isnull()]
    data.prior = data.prior.apply(checkPrior)
    data['phrase'] = data.prior.cumsum()
        
    verbosity("Processing data into phrase vectors", debug)
    verbosity("Step 1: Translating to indexes", debug)
    checkpoint = time.time()
    data['word_ix'] = data.token.apply(get_word_ix)
    data['case_ix'] = data.token.apply(get_casing_ix)
    data['label_ix'] = data.BIO.apply(lambda x: label_to_ix[x])
    verbosity(f"Step 1: Translated to indexes complete: {time.time()-checkpoint} s", debug)

    verbosity("Step 2: Creating phrase vectors", debug)
    verbosity("Step 2a: Aggregating phrases", debug)
    checkpoint = time.time()
    phrase_vectors = data.groupby('phrase').agg({'word_ix': list, 'case_ix': list, 'label_ix': list})
    verbosity(f"Step 2a: {time.time()-checkpoint} s", debug)
    
    verbosity("Step 2b: Padding word vectors", debug)
    checkpoint = time.time()
    phrase_vectors['word_vector'] = phrase_vectors.word_ix.apply(lambda x: pad_truncate(x, SENTENCE_WIDTH, word_to_ix[PAD_WORD]))
    verbosity(f"Step 2b: {time.time()-checkpoint} s", debug)
    
    verbosity("Step 2c: Creating and padding character vectors", debug)
    checkpoint = time.time()
    phrase_vectors['char_vector'] = phrase_vectors.word_vector.apply(lambda x: phrase2char(x))
    verbosity(f"Step 2c: {time.time()-checkpoint} s", debug)
    
    verbosity(f"Step 2d: Padding case vectors", debug)
    checkpoint = time.time()
    phrase_vectors['case_vector'] = phrase_vectors.case_ix.apply(lambda x: pad_truncate(x, SENTENCE_WIDTH, case_to_ix[CASE_PAD_SYMBOL]))
    verbosity(f"Step 2d: {time.time()-checkpoint}", debug)
    
    verbosity("Step 2e: Padding label vectors", debug)
    checkpoint = time.time()
    phrase_vectors['label_vector'] = phrase_vectors.label_ix.apply(lambda x: np.expand_dims(pad_truncate(x, SENTENCE_WIDTH, label_to_ix[LABEL_PAD_SYMBOL]), -1))
    verbosity(f"Step 2e: {time.time()-checkpoint} s", debug)
    
    verbosity("Saving data to disk", debug)
    checkpoint = time.time()
    phrase_vectors.drop(columns=['word_ix', 'case_ix', 'label_ix'], inplace=True)
    phrase_vectors = phrase_vectors.to_numpy()
    phrase_vectors = [np.stack(phrase_vectors[:,0]), np.stack(phrase_vectors[:,1]), np.stack(phrase_vectors[:,2]), np.stack(phrase_vectors[:,3])]
    
    # saving in multi parts because training data causes a memory error
    np.save(clean_name+'word', phrase_vectors[0], allow_pickle=True)
    np.save(clean_name+'char', phrase_vectors[1], allow_pickle=True)
    np.save(clean_name+'case', phrase_vectors[2], allow_pickle=True)
    np.save(clean_name+'label', phrase_vectors[3], allow_pickle=True)

    verbosity(f"Saved to disk: {time.time()-checkpoint} s", debug)
  
  return phrase_vectors

In [9]:
train_data = preprocess_data(train_file, use_cache=True)
print(train_data[0].shape,train_data[1].shape,train_data[2].shape,train_data[3].shape)

Attempting to load from cache
Loaded successfully
(115812, 256) (115812, 256, 52) (115812, 256) (115812, 256, 1)


## Prepare development data

In [10]:
dev_data = preprocess_data(dev_file, use_cache=True)
print(dev_data[0].shape,dev_data[1].shape,dev_data[2].shape,dev_data[3].shape)

Attempting to load from cache
Loaded successfully
(15680, 256) (15680, 256, 52) (15680, 256) (15680, 256, 1)


## Prepare test data

In [11]:
test_data = preprocess_data(test_file, use_cache=True)
print(test_data[0].shape,test_data[1].shape,test_data[2].shape,test_data[3].shape)

Attempting to load from cache
Loaded successfully
(12217, 256) (12217, 256, 52) (12217, 256) (12217, 256, 1)


# Model Building

In [0]:
# https://github.com/mxhofer/Named-Entity-Recognition-BidirectionalLSTM-CNN-CoNLL/blob/master/nn_CoNLL.ipynb

def buildModel(labels, wordEmbeddings, caseEmbeddings, characterEmbeddings=None):
  """Model layers"""
  
  # character input
  character_input = Input(shape=(None, config.word_length,), name="Character_input")
  embed_char_out = TimeDistributed(
      Embedding(config.char_vocab, config.char_embedding_dim, embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name="Character_embedding")(
      character_input)

  dropout = Dropout(config.dropout)(embed_char_out)

  # CNN
  conv1d_out = TimeDistributed(Conv1D(kernel_size=config.conv_size, filters=config.conv_filters, padding='same', activation='tanh', strides=config.conv_stride), name="Convolution")(dropout)
  maxpool_out = TimeDistributed(MaxPooling1D(config.conv_window), name="Maxpool")(conv1d_out)
  char = TimeDistributed(Flatten(), name="Flatten")(maxpool_out)
  char = Dropout(config.dropout)(char)

  # word-level input
  words_input = Input(shape=(None,), dtype='int32', name='words_input')
  words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1], weights=[wordEmbeddings],
                    trainable=False)(words_input)

  # case-info input
  casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
  casing = Embedding(input_dim=caseEmbeddings.shape[0], output_dim=caseEmbeddings.shape[1], weights=[caseEmbeddings],
                     trainable=False)(casing_input)
  
  # concat & BLSTM
  output = concatenate([words, casing, char])
  output = Bidirectional(LSTM(config.lstm_state_size, 
                              return_sequences=True, 
                              dropout=config.dropout,                        # on input to each LSTM block
                              recurrent_dropout=config.recurrent_dropout     # on recurrent input signal
                             ), name="BLSTM")(output)
  output = TimeDistributed(Dense(len(labels), activation='softmax'),name="Softmax_layer")(output)

  # set up model
  model = Model(inputs=[words_input, character_input, casing_input], outputs=[output])

  model.compile(loss='sparse_categorical_crossentropy', optimizer=Nadam())
  
  return model

In [0]:
myModel = None
if os.path.exists(model_load_path):
  print("Attempting to load model")
  myModel = load_model(model_load_path)
  print("Model loaded successfully")
  myModel.summary()
  myModel.fit([train_data[0][:config.training_size],train_data[1][:config.training_size],train_data[2][:config.training_size]], train_data[3][:config.training_size],
              validation_data = (dev_data[:3], dev_data[3]),
              epochs=config.epochs,
              initial_epoch=INITIAL_EPOCH,
              batch_size=config.batch_size,
              callbacks=[EarlyStopping(min_delta=0), ModelCheckpoint(model_path)])
  myModel.save(os.path.join(wandb.run.dir, "model.h5"))
else:
  print("Building model")
  myModel = buildModel(labels, words, cases)
  myModel.summary()
  myModel.fit([train_data[0][:config.training_size],train_data[1][:config.training_size],train_data[2][:config.training_size]], train_data[3][:config.training_size],
              validation_data = (dev_data[:3], dev_data[3]),
              epochs=config.epochs,
              initial_epoch=0,
              batch_size=config.batch_size,
              callbacks=[EarlyStopping(min_delta=0), ModelCheckpoint(model_path)])
  myModel.save(os.path.join(wandb.run.dir, "model.h5"))

W0727 17:19:33.942376 139775883548544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 17:19:33.973322 139775883548544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0727 17:19:33.979979 139775883548544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 17:19:34.013591 139775883548544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0727 17:19:34.024033 

Building model


W0727 17:19:34.162523 139775883548544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0727 17:19:37.512042 139775883548544 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0727 17:19:37.675576 139775883548544 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Character_input (InputLayer)    (None, None, 52)     0                                            
__________________________________________________________________________________________________
Character_embedding (TimeDistri (None, None, 52, 30) 3000        Character_input[0][0]            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, None, 52, 30) 0           Character_embedding[0][0]        
__________________________________________________________________________________________________
Convolution (TimeDistributed)   (None, None, 52, 30) 2730        dropout_1[0][0]                  
__________________________________________________________________________________________________
Maxpool (T

In [0]:
def get_metrics(model, data, save=True):
  predictions = model.predict(data[:3])
  y = data[3].reshape(data[3].shape[0],data[3].shape[1])
  
  pf = np.argmax(predictions, axis=2).flatten()
  af = data[3].flatten()
  
  metrics = []
  metrics = pd.DataFrame(columns=['Label', 'Support', 'Precision', 'Recall', "F1"])
  for i, label in enumerate(labels):
    support = np.where(af == i)
    tp = np.sum(pf[support] == af[support])

    precision = None
    if pf[np.where(pf == i)].shape[0] == 0:
      precision = 0.0
    else:
      precision = tp/pf[np.where(pf == i)].shape[0]
      
    recall = tp/af[support].shape[0]
    
    f1 = None
    if precision + recall == 0:
      f1 = 0
    else:
      f1 = 2*precision*recall/(precision+recall)

    metrics = metrics.append({'Label': ix_to_label[i], 'Support':af[support].shape[0], 'Precision': precision, 'Recall':recall, 'F1':f1}, ignore_index=True)
  
  metrics = metrics.append({'Label': 'micro',
                  'Support': metrics.Support.sum(),
                  'Precision': (metrics.Precision*metrics.Support/pf.shape[0]).sum(),
                  'Recall': (metrics.Recall*metrics.Support/pf.shape[0]).sum(),
                  'F1': (metrics.F1*metrics.Support/pf.shape[0]).sum()
                           },
                 ignore_index=True)
  metrics = metrics.set_index('Label')
  if save:
    metrics.to_csv(model_results_path)
  display(metrics)
  return predictions
 

In [0]:
pred = get_metrics(myModel, test_data)